In [1]:
import pandas as pd
import minsearch
from groq import Groq
from mistralai import Mistral
from dotenv import load_dotenv
import os

## Loading Environment Variables

In [2]:
load_dotenv()


True

In [3]:
# Access the API keys
mistral_api_key = os.getenv("MISTRAL_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
client = Groq(api_key=groq_api_key)
client_mistral=Mistral(api_key=mistral_api_key)

## Data Loading

In [5]:
df=pd.read_csv("../dataset/Mental_Health_FAQ.csv")

In [6]:
df.head(10)

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
5,1619387,What should I do if I know someone who appears...,Although this website cannot substitute for pr...
6,1030153,How can I find a mental health professional fo...,Feeling comfortable with the professional you ...
7,8022026,What treatment options are available?,Just as there are different types of medicatio...
8,1155199,"If I become involved in treatment, what do I n...",Since beginning treatment is a big step for in...
9,7760466,What is the difference between mental health p...,There are many types of mental health professi...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Question_ID  98 non-null     int64 
 1   Questions    98 non-null     object
 2   Answers      98 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [8]:
documents=df.to_dict('records')

In [9]:
documents[0]

{'Question_ID': 1590140,
 'Questions': 'What does it mean to have a mental illness?',
 'Answers': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condit

## Indexing the data Using Minsearch

In [10]:
!curl -O https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3832  100  3832    0     0   4320      0 --:--:-- --:--:-- --:--:--  4344
100  3832  100  3832    0     0   4318      0 --:--:-- --:--:-- --:--:--  4344


In [11]:
df

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
...,...,...,...
93,4373204,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...
94,7807643,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c..."
95,4352464,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...
96,6521784,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...


In [12]:
df = df.rename(columns={'Question_ID': 'question_id', 'Questions': 'questions','Answers':'answers'})


In [13]:
df.columns

Index(['question_id', 'questions', 'answers'], dtype='object')

In [14]:
documents=df.to_dict('records')

In [15]:
documents[0]

{'question_id': 1590140,
 'questions': 'What does it mean to have a mental illness?',
 'answers': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condit

In [16]:
index=minsearch.Index(
    text_fields=['questions', 'answers'],
    keyword_fields=[]
)

In [17]:
index.fit(documents)

In [18]:
query="What should I eat if I lost a friend"

In [122]:
index.search(query)

## Evaluating Retrieval

In [20]:
df_questions = pd.read_csv('../dataset/ground_truth_data.csv')

In [21]:
df_questions.head()

,id,question
0,1590140,How do mental illnesses affect a person's dail...
1,1590140,What are some examples of serious mental illne...
2,1590140,Why is it a misconception to associate mental ...
3,1590140,"How are mental illnesses treated, and what is ..."
4,1590140,Can a person with mental illness become indepe...


In [22]:
ground_truth=df_questions.to_dict('records')

In [23]:
ground_truth[0]

{'id': 1590140,
 'question': "How do mental illnesses affect a person's daily functioning and relationships?"}

In [24]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [25]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


In [26]:
def precision(relevance_list):
    """
    Precision: Proportion of retrieved documents that are relevant.
    """
    relevant_retrieved = sum(relevance_list)  # True indicates relevance
    total_retrieved = len(relevance_list)  # All retrieved documents
    if total_retrieved == 0:
        return 0.0  # Avoid division by zero
    return relevant_retrieved / total_retrieved

In [27]:
def recall(relevance_list, total_relevant):
    """
    Recall: Proportion of relevant documents that are retrieved.
    """
    relevant_retrieved = sum(relevance_list)  # True indicates relevance
    if total_relevant == 0:
        return 0.0  # Avoid division by zero
    return relevant_retrieved / total_relevant

In [28]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    precision_scores = []
    recall_scores = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        
        # Get search results for this query
        results = search_function(q)
        
        # Check if the correct document (matching question_id) is in the results
        relevance = [doc['question_id'] == doc_id for doc in results]
        relevance_total.append(relevance)
        
        # Precision: Fraction of retrieved documents that are relevant
        precision_score = precision(relevance)
        precision_scores.append(precision_score)
        
        # Recall: There is only 1 relevant document per query, so recall is either 1 or 0
        recall_score = recall(relevance, 1)
        recall_scores.append(recall_score)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
        'precision': sum(precision_scores) / len(precision_scores),  # Average precision
        'recall': sum(recall_scores) / len(recall_scores),  # Average recall
    }


In [29]:
from tqdm.auto import tqdm


In [30]:
evaluate(ground_truth, lambda q: search(q['question']))


  0%|          | 0/366 [00:00<?, ?it/s]

{'hit_rate': 0.9180327868852459,
 'mrr': 0.7107739179460493,
 'precision': 0.09180327868852516,
 'recall': 0.9180327868852459}

High Recall, Low Precision: This suggests that the system is finding most of the relevant documents, but it is also retrieving a lot of irrelevant ones. You might want to focus on improving the precision by better tuning your search algorithm (e.g., improving query relevance, adjusting boosts, or filtering irrelevant results).

## Rag Flow

In [68]:
prompt_template="""
You are an expert mental health assistant specialized in providing detailed and accurate answers based on the given context.
Answer the QUESTION based on the CONTEXT from our meantal health database.
Use only the facts from the CONTEXT when answering the QUESTION.

Here is the context:

Context: {context}

Please answer the following question based on the provided context:

Question: {question}

Provide a detailed and informative response. Ensure that your answer is clear, concise, and directly addresses the question while being relevant to the context provided.

Your response should be in plain text and should not include any code blocks or extra formatting.

Answer:
""".strip()
def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [132]:
def llm(prompt,model):
        response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}])
        return response.choices[0].message.content

In [70]:
def rag(query, model='mixtral-8x7b-32768'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [ ]:
query="What should I eat if I lost a friend"

In [71]:
rag(query)

'I\'m really sorry to hear that you\'re going through a tough time. Losing a friend can be very difficult and may cause emotional distress. While I can\'t replace the support of a friend or provide mental health advice, I can offer some general suggestions about comforting self-care, including nutrition, during this time.\n\nIt\'s important to remember that there isn\'t a specific "food" to eat when coping with the loss of a friend. However, consuming a balanced and nutritious diet can generally contribute to overall well-being and help manage stress. Here are some ideas:\n\n1. Include a variety of fruits and vegetables in your meals and snacks. These foods are packed with essential vitamins and minerals as well as dietary fiber, which can aid in digestion.\n\n2. Choose whole grains like brown rice, oatmeal, and whole-wheat bread. They provide long-lasting energy, support healthy blood sugar levels, and are rich in B vitamins that can help with stress management.\n\n3. Consume lean pro

## Rag Evaluation

In [64]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()


In [41]:
len(ground_truth)

366

In [43]:
ground_truth[0]

{'id': 1590140,
 'question': "How do mental illnesses affect a person's daily functioning and relationships?"}

In [44]:
record=ground_truth[0]
question=record["question"]
answer_llm=rag(question)

In [65]:
prompt=prompt2_template.format(question=question,answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: How do mental illnesses affect a person's daily functioning and relationships?
Generated Answer: Mental illnesses can significantly impact a person's daily functioning and relationships in various ways. According to the context provided, mental health conditions may affect an individual's ability to perform daily tasks, maintain social connections, and manage their emotions.

Firstly, mental illnesses can make it challenging for a person to carry out routine activities. They might struggle with concentrating on work or school tasks, leading to reduced productivity and potential job loss or academic difficulties. Additionally, mental health issues can cause problems with time management, decisi

In [77]:
from tqdm.auto import tqdm
import json

evaluations=[]

for record in tqdm(ground_truth):
    id=record['id']
    if id in evaluations:
        continue
    question=record['question']
    answer_llm=rag(question)
    prompt=prompt2_template.format(
        question=question,
        answer_llm=answer_llm)
    evaluation=llm(prompt,model='mixtral-8x7b-32768')
    evaluation = json.loads(evaluation)
    evaluations.append((record,answer_llm,evaluation))
    
    

  0%|          | 0/366 [00:00<?, ?it/s]

In [79]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])


In [80]:
df_eval.head()

,record,answer,evaluation
0,"{'id': 1590140, 'question': 'How do mental ill...",Mental illnesses can significantly affect a pe...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
1,"{'id': 1590140, 'question': 'What are some exa...","Sure, based on the context provided, some exam...","{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
2,"{'id': 1590140, 'question': 'Why is it a misco...",Mental illness is not a reflection of a person...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
3,"{'id': 1590140, 'question': 'How are mental il...",Mental illnesses are treated using a variety o...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
4,"{'id': 1590140, 'question': 'Can a person with...","Yes, a person with a mental illness can certai...","{'Relevance': 'RELEVANT', 'Explanation': 'The ..."


In [86]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [89]:
df_eval.head()

,id,answer,question,relevance,explanation
0,1590140,Mental illnesses can significantly affect a pe...,How do mental illnesses affect a person's dail...,RELEVANT,The generated answer is relevant to the questi...
1,1590140,"Sure, based on the context provided, some exam...",What are some examples of serious mental illne...,RELEVANT,The generated answer provides three clear exam...
2,1590140,Mental illness is not a reflection of a person...,Why is it a misconception to associate mental ...,RELEVANT,The generated answer is highly relevant to the...
3,1590140,Mental illnesses are treated using a variety o...,"How are mental illnesses treated, and what is ...",RELEVANT,The generated answer is relevant as it explain...
4,1590140,"Yes, a person with a mental illness can certai...",Can a person with mental illness become indepe...,RELEVANT,The generated answer directly addresses the qu...


In [91]:
# Count the occurrences of each relevance category
relevance_counts = df_eval['relevance'].value_counts()
print(relevance_counts)


relevance
RELEVANT           346
PARTLY_RELEVANT     18
NON_RELEVANT         1
RELEVELANT           1
Name: count, dtype: int64


In [126]:
import matplotlib.pyplot as plt
import seaborn as sns

# Bar plot for relevance counts
plt.figure(figsize=(8, 5))
sns.countplot(data=df_eval, x='relevance', palette='viridis')
plt.title('Count of Relevance Categories')
plt.xlabel('Relevance')
plt.ylabel('Count')
plt.show()


ModuleNotFoundError: No module named 'seaborn'

In [100]:
!pipenv install seaborn

Installing seaborn...

Loading .env environment variables...



Resolving seaborn...
[    ] Installing...
Installation Succeeded
[    ] Installing seaborn...
[    ] Installing seaborn...

Installing dependencies from Pipfile.lock (153417)...


In [123]:
# Filter relevant entries
#relevant_answers = df_eval[df_eval['relevance'] == 'RELEVANT']
relevant_answers.head(5)


,id,answer,question,relevance,explanation
0,1590140,"Mental illnesses can significantly affect a person's daily functioning and relationships in various ways. \n\nIn terms of daily functioning, mental illnesses can cause difficulties with concentration, memory, and decision-making, making it challenging to perform routine tasks or handle responsibilities at work or school. They can also lead to changes in sleep patterns, appetite, and energy levels, which can impact a person's ability to maintain a regular schedule or engage in daily activities. Furthermore, mental illnesses can result in physical symptoms such as fatigue, headaches, and stomachaches, which can exacerbate functional impairments.\n\nRegarding relationships, mental illnesses can affect a person's ability to communicate effectively, leading to misunderstandings or conflicts with others. They can also cause changes in mood and behavior that may be perceived as unpredictable or unreliable, making it difficult to form and maintain close relationships. In some cases, mental illnesses may lead to social isolation or withdrawal due to feelings of shame, stigma, or fear of rejection.\n\nAdditionally, mental illnesses can have a ripple effect on the relationships of family members and friends, who may experience stress, frustration, and burden as they try to support their loved one. This can lead to strained relationships or even family dysfunction if left unaddressed.\n\nOverall, mental illnesses can have far-reaching impacts on a person's daily functioning and relationships, highlighting the importance of seeking appropriate treatment and support.",How do mental illnesses affect a person's daily functioning and relationships?,RELEVANT,"The generated answer is relevant to the question as it provides detailed information about how mental illnesses can affect a person's daily functioning and relationships. It covers various aspects such as cognitive difficulties, changes in sleep patterns, communication problems, mood changes, and the impact on family and friends. The answer is comprehensive and directly addresses the question posed."
1,1590140,"Sure, based on the context provided, some examples of serious mental illnesses that can result in major functional impairment include Schizophrenia, Bipolar Disorder, and Major Depressive Disorder.\n\n1. Schizophrenia: This is a chronic and severe mental disorder that affects how a person thinks, feels, and behaves. People with schizophrenia may have difficulty distinguishing reality from fantasy, which can lead to hallucinations, delusions, and paranoia. This can significantly impair their ability to function in daily life, including work, school, and social relationships.\n\n2. Bipolar Disorder: This is a mental health condition that causes extreme mood swings that include emotional highs (manic or hypomanic episodes) and lows (depressive episodes). These mood swings can significantly affect a person's energy, activity, judgment, behavior, and ability to carry out day-to-day tasks.\n\n3. Major Depressive Disorder: This is a severe form of depression that can prevent a person from functioning normally. It can cause persistent feelings of sadness, hopelessness, and a lack of interest or pleasure in activities. It can also lead to significant weight loss or gain, insomnia or oversleeping, fatigue, feelings of worthlessness or guilt, and difficulty thinking, concentrating, or making decisions.\n\nThese mental illnesses can be managed with appropriate treatment, which may include medication, therapy, or a combination of both. Early identification and treatment can help reduce the impact of these illnesses on a person's ability to function in daily life.",What are some examples of serious mental illnesses that can result in major functional impairment?,RELEVANT,"The generated answer provides three clear examples of serious mental illnesses that can result in major functional impairment, which directly answers the question. The answer 

In [118]:
# Filter non-relevant entries
non_relevant_answers = df_eval[df_eval['relevance'] == 'NON_RELEVANT']


In [117]:
non_relevant_answers[['question','answer','explanation']]

,question,answer,explanation
32,What kind of physical or neurological tests are usually involved?,"The context does not provide specific information about what kind of physical or neurological tests are usually involved in the assessment and treatment of mental health conditions. The text mentions that mental health conditions can be caused by a combination of genetic, biological, environmental, and psychological factors. However, it does not outline any specific tests used to identify these factors. Therefore, I cannot provide a detailed and accurate answer to this question based on the given context.","The generated answer does not provide any information about the types of physical or neurological tests involved in mental health assessments, therefore it is non-relevant to the question asked."


In [116]:
import pandas as pd

# Set display options to show full content
pd.set_option('display.max_colwidth', None)


## Analysing Partly Relevant Answers

In [119]:
partly_relevant_answers = df_eval[df_eval['relevance'] == 'PARTLY_RELEVANT']


In [120]:
partly_relevant_answers[['question','answer','explanation']]

,question,answer,explanation
5,Who are the groups most commonly affected by different mental illnesses?,"The context provided does not include specific information about the groups most commonly affected by different mental illnesses. However, I can share some general information based on common knowledge in the field of mental health.\n\nVarious factors such as genetics, environment, and lifestyle can contribute to the development of mental illnesses. While mental illnesses can affect anyone, regardless of their background, some groups may be at a higher risk due to specific factors. Here are some examples:\n\n1. Women are more likely to experience depression and anxiety disorders than men, potentially due to hormonal differences, societal pressures, and a higher likelihood of trauma exposure.\n2. Men have higher suicide rates and are more likely to experience substance use disorders, possibly due to societal expectations, stress, and a lower likelihood of seeking help for mental health issues.\n3. Older adults are at risk for mental health issues such as depression, anxiety, and dementia, which can be caused by age-related changes in the brain, chronic illnesses, and social isolation.\n4. Children and adolescents are susceptible to mental health issues like ADHD, autism spectrum disorders, and depression, which can be influenced by genetic factors, brain development, and environmental factors such as bullying or family dysfunction.\n5. Individuals with lower socioeconomic status or from marginalized communities may experience higher rates of mental health issues due to discrimination, trauma, and limited access to mental health services.\n\nIt's important to note that these are general trends and individual experiences can vary greatly. Mental health issues can affect anyone, regardless of their age, gender, ethnicity, or socioeconomic status.","The generated answer is not directly relevant to the question as it does not specify the groups most commonly affected by different mental illnesses based on the provided context. However, the answer does provide general information about various groups that are at a higher risk of mental illnesses due to specific factors, making it partly relevant."
9,At what age are individuals most susceptible to mental illness?,"The context does not provide specific information about the age at which individuals are most susceptible to mental illness. However, it's important to note that mental illness can affect people of all ages, from children to older adults. According to the World Health Organization, half of all mental illnesses begin by the age of 14, and three-quarters of mental illnesses begin by the age of 24. Early intervention and treatment can significantly improve outcomes, so it's crucial to identify and address mental health concerns as early as possible.","While the generated answer does not provide a specific age at which individuals are most susceptible to mental illness, it does offer relevant information about the onset of mental illness. According to the answer, half of all mental illnesses begin by the age of 14 and three-quarters of mental illnesses begin by the age of 24. This context helps illustrate that mental illness can affect people at varying ages, which is pertinent to the question."
30,What can I expect in terms of the cause of my illness?,"I'm glad you're seeking information about the potential causes of your illness. However, I'm unable to provide a specific answer because the context provided does not include any details about your diagnosis or symptoms. \n\nGenerally, mental health conditions can be caused by a variety of factors including genetic, biological, environmental, and psychological influences. For example, depression can be caused by a combination of genetic predisposition, brain chemistry imbalances, traumatic events, and chronic stress. Anxiety disorders can be caused by a combination of genetic factors, brain abnormalities, environmental stresso

In [121]:
df_eval.to_csv('../dataset/rag-eval-mistral.csv', index=False)


## Evaluating Mistralai Model

In [135]:
evaluations_llama = []

for record in tqdm(ground_truth):
    question = record['question']
    answer_llm = rag(question, model='llama3-70b-8192') 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt,model='llama3-70b-8192')
    evaluation = json.loads(evaluation)
    
    evaluations_llama.append((record, answer_llm, evaluation))

  0%|          | 0/366 [00:00<?, ?it/s]

In [137]:
df_eval_llama = pd.DataFrame(evaluations_llama, columns=['record', 'answer', 'evaluation'])

df_eval_llama['id'] = df_eval_llama.record.apply(lambda d: d['id'])
df_eval_llama['question'] = df_eval_llama.record.apply(lambda d: d['question'])

df_eval_llama['relevance'] = df_eval_llama.evaluation.apply(lambda d: d['Relevance'])
df_eval_llama['explanation'] = df_eval_llama.evaluation.apply(lambda d: d['Explanation'])

del df_eval_llama['record']
del df_eval_llama['evaluation']

In [140]:
df_eval_llama.head(3)

,answer,id,question,relevance,explanation
0,"Based on the provided context, it is clear that mental illnesses can have a significant impact on a person's daily functioning and relationships.\n\nMental illnesses can affect a person's daily functioning in various ways. For instance, they can disrupt a person's ability to perform daily tasks, such as maintaining personal hygiene, cooking, or managing their finances. They can also impair cognitive functioning, leading to difficulties in concentrating, making decisions, or completing tasks. Additionally, mental illnesses can impact a person's sleep patterns, appetite, and energy levels, further compromising their daily functioning.\n\nIn terms of relationships, mental illnesses can lead to social withdrawal, making it challenging for individuals to maintain social connections and relationships. They may experience feelings of guilt, shame, or anxiety that prevent them from engaging with friends and family. Mental illnesses can also impact communication skills, leading to misunderstandings, conflicts, or difficulties in expressing emotions. Furthermore, they can strain relationships due to mood swings, irritability, or emotional reactivity.\n\nIt is essential to recognize that mental illnesses can vary in severity, and the impact on daily functioning and relationships can differ from person to person. However, with proper treatment, support, and self-care, individuals can learn to manage their symptoms and improve their overall well-being, leading to a more fulfilling daily life and healthier relationships.",1590140,How do mental illnesses affect a person's daily functioning and relationships?,RELEVANT,The generated answer directly addresses the question by providing specific examples of how mental illnesses can affect a person's daily functioning and relationships. It provides a comprehensive and well-structured response that fully covers the scope of the question.
1,"Based on the context provided, some examples of serious mental illnesses that can result in major functional impairment are:\n\n* Schizophrenia: A chronic and severe mental disorder that affects how a person thinks, feels, and behaves, often accompanied by hallucinations, delusions, and disorganized speech or behavior.\n\n* Bipolar Disorder: A mood disorder characterized by extreme shifts in mood, energy, and activity levels, ranging from severe depression to manic highs, which can impact daily functioning and relationships.\n\n* Major Depressive Disorder (MDD): A severe mental health disorder marked by persistent feelings of sadness, hopelessness, and loss of interest in activities, leading to significant impairment in social, occupational, or educational functions.\n\n* Post-Traumatic Stress Disorder (PTSD): A condition that develops after a person experiences a traumatic event, leading to symptoms such as flashbacks, nightmares, and avoidance of reminders of the event, which can result in significant impairment in daily life and relationships.\n\n* Obsessive-Compulsive Disorder (OCD): A mental health disorder characterized by recurring, intrusive thoughts and compulsions to perform repetitive behaviors, which can interfere with daily functioning, social relationships, and overall well-being.\n\nThese serious mental illnesses can significantly impact a person's ability to function in daily life, maintain relationships, and engage in productive activities, leading to major functional impairment if left untreated or undertreated.",1590140,What are some examples of serious mental illnesses that can result in major functional impairment?,RELEVANT,"The generated answer provides a list of serious mental illnesses that can result in major functional impairment, directly addressing the question. The answer is comprehensive, well-structured, and provides clear explanations for each example, making it highly relevant to the question."
2,"It is a misconception to associate mental illness with a person's character, intelligence

In [141]:
df_eval_llama.to_csv('../dataset/rag_eval_llama.csv', index=False)
